In [1]:
import numpy as np # type: ignore
import time
import torch
import torch.nn as nn # type: ignore
import torch.optim as optim # type: ignore
from torch.utils.data import random_split, DataLoader # type: ignore
from torch.optim.lr_scheduler import MultiStepLR
from chess import pgn # type: ignore
from tqdm import tqdm # type: ignore
from datetime import datetime
from auxiliary_func import check_memory, load_dataset, encode_moves
from dataset import ChessDataset
from model7 import ChessModel7
from MiniMaia import MiniMaia
import pickle
from auxiliary_func import load_pgn

In [5]:
with open(f"/home/wyatt/Documents/capstone/models/flipped_board_data_move_to_int", "rb") as file:
    move_to_int = pickle.load(file)

print(len(move_to_int))

1932


In [ ]:
data = load_dataset("/home/wyatt/Documents/capstone/chess-engine/data/Lichess_Elite_Database", 20)


 11%|█         | 6/56 [00:00<00:00, 51.63it/s]

. . . . . . . .
. . . . R . . .
. . k . . p . P
. . . . . . . .
. . . . . . n .
. . . . . . P .
. . . . . . . .
. . . . . . . K
. . . . . . . .
. . . . R . . .
. . k . . p . n
. . . . . . . .
. . . . . . . .
. . . . . . P .
. . . . . . . .
. . . . . . . K
. . . . . . . .
. . . . . . . .
. . k . R p . n
. . . . . . . .
. . . . . . . .
. . . . . . P .
. . . . . . . .
. . . . . . . K
. . . . . . . .
. . . . . . . .
. . . . R p . n
. . . k . . . .
. . . . . . . .
. . . . . . P .
. . . . . . . .
. . . . . . . K
. . . . . . . .
. . . . . . . .
. . . . . R . n
. . . k . . . .
. . . . . . . .
. . . . . . P .
. . . . . . . .
. . . . . . . K
. . . . . . . .
. . . . . . . .
. . . . . R . .
. . . k . . . .
. . . . . . n .
. . . . . . P .
. . . . . . . .
. . . . . . . K
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . k . . . .
. . . . . R n .
. . . . . . P .
. . . . . . . .
. . . . . . . K
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . k . . . .
. . . . . R . .
. . . . n . P .
. . . . 

 18%|█▊        | 10/56 [00:02<00:09,  4.88it/s]


In [ ]:
board = game.board()
print(board)
i = 0

headers = game.headers
print(headers["Termination"])
print(game.end())
print(game.headers)
# for move in game.mainline_moves():
#     i += 1
#     board.push(move)
#     print(board)
#     print("\n")
#     if i > 60:
#         time.sleep(1)

In [ ]:
from auxiliary_func import create_input_for_nn, load_pgn

create_input_for_nn()




In [ ]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MiniMaia(1000).to(device)
print(model)

In [ ]:
total_mem = check_memory()
print(total_mem)


X, y, games_parsed, files_parsed = load_dataset(data_folder="../../data/Lichess_Elite_Database", pgn_memory_mark=1.0, file_limit=2)


X, y = np.array(X, dtype=np.float32), np.array(y)

y, move_to_int = encode_moves(y)
num_classes = len(move_to_int)

In [ ]:

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)


In [ ]:
X.shape
# y.shape

In [ ]:
# Create Dataset
dataset = ChessDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Compute split sizes
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Then create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}', flush=True)

# Model Initialization
model = ChessModel7(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

scheduler = MultiStepLR(optimizer, milestones=[50000, 250000, 400000], gamma=0.2)

In [ ]:
model.train()
running_loss = 0.0
for inputs, labels in tqdm(train_loader):
    inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
    optimizer.zero_grad()

    outputs = model(inputs)  # Raw logits

    print(outputs)
    # Compute loss
    loss = criterion(outputs, labels)
    loss.backward()
    
    # Gradient clipping
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    optimizer.step()
    scheduler.step()
    print(loss.item())
    print(loss.item())
    running_loss += loss.item()